<a href="https://colab.research.google.com/github/amanbagrecha/datascience_project/blob/master/ZINDI_SPOT_THE_CROP_CHALLENGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://radiant-assets.s3-us-west-2.amazonaws.com/PrimaryRadiantMLHubLogo.png' alt='Radiant MLHub Logo' width='200'/>

# Top 5 solution for the Radiant Earth Spot the Crop Challenge

This notebook walks you through the steps to load the data and build a Top 5 solution model using Random Forests and Xgboost for `Radiant Earth Spot the Crop Challenge`. 

The notebook builds on top of baseline notebook provided by Radiant Earth

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Radiant MLHub API


The Radiant MLHub API gives access to open Earth imagery training data for machine learning applications. You can learn more about the repository at the [Radiant MLHub site](https://mlhub.earth) and about the organization behind it at the [Radiant Earth Foundation site](https://radiant.earth).

Full documentation for the API is available at [docs.mlhub.earth](docs.mlhub.earth).

Each item in our collection is explained in json format compliant with [STAC](https://stacspec.org/) [label extension](https://github.com/radiantearth/stac-spec/tree/master/extensions/label) definition.

## Dependencies



**You must replace the `YOUR_API_KEY_HERE` text with your API key which you can obtain by creating a free account on the [MLHub Dashboard](https://dashboard.mlhub.earth/) within the `API Keys` tab at the top of the page.**

In [2]:
!pip install radiant_mlhub
!pip install rasterio

     |████████████████████████████████| 133 kB 8.0 MB/s 
     |████████████████████████████████| 72 kB 769 kB/s 
     |████████████████████████████████| 61 kB 7.8 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.0
    Uninstalling tqdm-4.62.0:
      Successfully uninstalled tqdm-4.62.0
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.25.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 19.3 MB 1.2 MB/s 


In [3]:
from radiant_mlhub import Collection
import tarfile
import os
from pathlib import Path
import json

import datetime
import rasterio
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
import xgboost as xgb
import random
import gc

In [4]:
random.seed(10)

## Downloading and Loading the Data

In this part, we will download the data from Radiant MLHub and load the properties of each item in the dataset into a DataFrame


In [5]:
os.environ['MLHUB_API_KEY'] = 'YOUR_API_KEY_HERE'

collections = [
    'ref_south_africa_crops_competition_v1_train_labels',
    'ref_south_africa_crops_competition_v1_train_source_s1', 
    'ref_south_africa_crops_competition_v1_test_labels',
    'ref_south_africa_crops_competition_v1_test_source_s1', 
#     'ref_south_africa_crops_competition_v1_test_source_s2' # Uncomment this out if you want to download the Sentinel-2 Data (not needed for the Hackathon)
#     'ref_south_africa_crops_competition_v1_train_source_s2', # Uncomment this out if you want to download the Sentinel-2 Data (not needed for the Hackathon)

]

def download(collection_id):
    print(f'Downloading {collection_id}...')
    collection = Collection.fetch(collection_id)
    path = collection.download('.')
    path = collection_id + '.tar.gz'
    tar = tarfile.open(path, "r:gz")
    tar.extractall()
    tar.close()
    os.remove(path)
    
def resolve_path(base, path):
    return Path(os.path.join(base, path)).resolve()
    
def load_df(collection_id):
    collection = json.load(open(f'{collection_id}/collection.json', 'r'))
    rows = []
    item_links = []
    for link in collection['links']:
        if link['rel'] != 'item':
            continue
        item_links.append(link['href'])
        
    for item_link in item_links:
        item_path = f'{collection_id}/{item_link}'
        current_path = os.path.dirname(item_path)
        item = json.load(open(item_path, 'r'))
        tile_id = item['id'].split('_')[-1]
        for asset_key, asset in item['assets'].items():
            rows.append([
                tile_id,
                None,
                None,
                asset_key,
                str(resolve_path(current_path, asset['href']))
            ])
            
        for link in item['links']:
            if link['rel'] != 'source':
                continue
            link_path = resolve_path(current_path, link['href'])
            source_path = os.path.dirname(link_path)
            try:
                source_item = json.load(open(link_path, 'r'))
            except FileNotFoundError:
                continue
            datetime = source_item['properties']['datetime']
            satellite_platform = source_item['collection'].split('_')[-1]
            for asset_key, asset in source_item['assets'].items():
                rows.append([
                    tile_id,
                    datetime,
                    satellite_platform,
                    asset_key,
                    str(resolve_path(source_path, asset['href']))
                ])
    return pd.DataFrame(rows, columns=['tile_id', 'datetime', 'satellite_platform', 'asset', 'file_path'])

for c in collections:
    download(c)



  0%|          | 0/31.4 [00:00<?, ?M/s]

  0%|          | 0/5987.8 [00:00<?, ?M/s]

  0%|          | 0/10.9 [00:00<?, ?M/s]

  0%|          | 0/2566.1 [00:00<?, ?M/s]

In [6]:
competition_train_df = load_df('ref_south_africa_crops_competition_v1_train_labels')
competition_test_df = load_df('ref_south_africa_crops_competition_v1_test_labels')

In [7]:
# This DataFrame lists all types of assets including documentation of the data. 
# In the following, we will use the Sentinel-1 bands (VV and VH) as well as labels. 
competition_train_df['asset'].unique()

array(['documentation', 'field_ids', 'field_info_train', 'labels',
       'raster_values', 'VH', 'VV'], dtype=object)

In [8]:
competition_train_df.head()

,tile_id,datetime,satellite_platform,asset,file_path
0,2587,None,None,documentation,/content/ref_south_africa_crops_competition_v1...
1,2587,None,None,field_ids,/content/ref_south_africa_crops_competition_v1...
2,2587,None,None,field_info_train,/content/ref_south_africa_crops_competition_v1...
3,2587,None,None,labels,/content/ref_south_africa_crops_competition_v1...
4,2587,None,None,raster_values,/content/ref_south_africa_crops_competition_v1...


In [9]:
# find unique tiles id
tile_ids_train = competition_train_df['tile_id'].unique()

In [10]:
# calculate the distribution of tiles time-stamp  
cdf={}
for tile_id in tile_ids_train:
    if tile_id != '1951': # avoid using this specific tile for the Hackathon as it might have a missing file
        
        tile_df = competition_train_df[competition_train_df['tile_id']==tile_id]
        tile_date_times = tile_df[tile_df['satellite_platform']=='s1']['datetime'].unique()
        cdf[len(tile_date_times)] = cdf.get(len(tile_date_times),0)+1 

In [11]:
cdf

{20: 5, 21: 1713, 23: 2, 33: 1, 37: 2, 40: 1, 41: 925}

In [12]:
# loop over first 1000 tiles
X = np.empty((0, 14), np.uint8)
y = np.empty((0, 1), np.uint8)
field_ids = np.empty((0, 1), np.uint32)

for tile_id in tile_ids_train[:1000]:
    if tile_id != '1951': # avoid using this specific tile for the Hackathon as it might have a missing file
        
        tile_df = competition_train_df[competition_train_df['tile_id']==tile_id]

        field_id_src = rasterio.open(tile_df[tile_df['asset']=='field_ids']['file_path'].values[0])
        field_id_array = field_id_src.read(1).flatten()
        my_idx = np.where(field_id_array!=0)[0]
        field_ids = np.append(field_ids, field_id_array[my_idx])
        
        label_src = rasterio.open(tile_df[tile_df['asset']=='labels']['file_path'].values[0])
        label_array = label_src.read(1).flatten()
        y = np.append(y, label_array[my_idx])


        tile_date_times = tile_df[tile_df['satellite_platform']=='s1']['datetime'].unique()
        X_tile = np.empty((len(my_idx), 0), np.uint8)
        if(len(tile_date_times))>36:
          for date_time in tile_date_times[ :40:6]:
              vv_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='VV')]['file_path'].values[0])
              vv_array = np.expand_dims(vv_src.read(1).flatten()[my_idx], axis=1)

              vh_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='VH')]['file_path'].values[0])
              vh_array = np.expand_dims(vh_src.read(1).flatten()[my_idx], axis=1)

              X_tile = np.append(X_tile, vv_array, axis = 1)
              X_tile = np.append(X_tile, vh_array, axis = 1)
        else:

          for date_time in tile_date_times[ :20:3]:
              vv_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='VV')]['file_path'].values[0])
              vv_array = np.expand_dims(vv_src.read(1).flatten()[my_idx], axis=1)

              vh_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='VH')]['file_path'].values[0])
              vh_array = np.expand_dims(vh_src.read(1).flatten()[my_idx], axis=1)

              X_tile = np.append(X_tile, vv_array, axis = 1)
              X_tile = np.append(X_tile, vh_array, axis = 1)

        X = np.append(X, X_tile, axis=0)

In [ ]:
# loop over last ~1600 tiles and append to prev X array
for tile_id in tile_ids_train[1000:]:
    if tile_id != '1951': # avoid using this specific tile for the Hackathon as it might have a missing file
        
        tile_df = competition_train_df[competition_train_df['tile_id']==tile_id]

        field_id_src = rasterio.open(tile_df[tile_df['asset']=='field_ids']['file_path'].values[0])
        field_id_array = field_id_src.read(1).flatten()
        my_idx = np.where(field_id_array!=0)[0]
        field_ids = np.append(field_ids, field_id_array[my_idx])
        
        label_src = rasterio.open(tile_df[tile_df['asset']=='labels']['file_path'].values[0])
        label_array = label_src.read(1).flatten()
        y = np.append(y, label_array[my_idx])


        tile_date_times = tile_df[tile_df['satellite_platform']=='s1']['datetime'].unique()
        X_tile = np.empty((len(my_idx), 0), np.uint8)
        if(len(tile_date_times))>36:
          for date_time in tile_date_times[ :40:6]:
              vv_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='VV')]['file_path'].values[0])
              vv_array = np.expand_dims(vv_src.read(1).flatten()[my_idx], axis=1)

              vh_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='VH')]['file_path'].values[0])
              vh_array = np.expand_dims(vh_src.read(1).flatten()[my_idx], axis=1)

              X_tile = np.append(X_tile, vv_array, axis = 1)
              X_tile = np.append(X_tile, vh_array, axis = 1)
        else:

          for date_time in tile_date_times[ :20:3]:
              vv_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='VV')]['file_path'].values[0])
              vv_array = np.expand_dims(vv_src.read(1).flatten()[my_idx], axis=1)

              vh_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='VH')]['file_path'].values[0])
              vh_array = np.expand_dims(vh_src.read(1).flatten()[my_idx], axis=1)

              X_tile = np.append(X_tile, vv_array, axis = 1)
              X_tile = np.append(X_tile, vh_array, axis = 1)

        X = np.append(X, X_tile, axis=0)

In [14]:
X.shape

(65780218, 14)

In [15]:
data = pd.DataFrame(X)
data['label'] = y
data['field_id'] = field_ids
data = data[data.field_id != 0]
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,label,field_id
0,13,3,9,1,37,3,33,3,29,4,10,5,18,5,2,3020
1,16,1,5,2,21,1,16,1,29,4,18,4,31,3,2,3020
2,11,2,6,0,20,3,16,2,21,3,25,3,33,5,2,3020
3,6,2,6,1,32,3,30,3,25,8,35,4,48,4,2,3020
4,11,2,7,0,20,3,16,2,22,1,22,5,29,7,2,3020


In [ ]:
del X
gc.collect()

209

## Building the Model

In [16]:
class group_by_attribute(object):
    """groub by field id and calculate mean on incoming data"""
    def __init__(self):
        super(group_by_attribute, self).__init__()
        self.gb = np.empty((0,1), np.uint8)

    def compute(self,df, idx):
        self.gb = df.loc[:,['field_id', idx]].groupby('field_id').mean()
        if(idx=='label'):
            self.gb = self.gb.round(0).astype(int)

In [17]:
# create objects for each cloumn to perform group-by 
gb_objs = [group_by_attribute() for i in range(len(data.columns)-1) ]

In [19]:
# perform group-by on individual column by passing column name and dataframe
for i,j in enumerate(data.columns[:-1]):
  gb_objs[i].compute(data,j)

In [20]:
# perform concatination on field_id
from functools import reduce 
df_final = [i.gb for i in gb_objs]

df_merged = reduce(lambda left,right: pd.merge(left,right,on='field_id'), df_final)

In [21]:
df_merged = df_merged.reset_index()

In [22]:
df_merged.head()

,field_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,label
0,1,14.143087,2.979100,11.432476,2.872990,27.040193,7.993569,19.217042,4.901929,22.573955,6.652733,22.879421,7.479100,21.461415,5.821543,4
1,2,10.521739,2.913043,7.478261,2.956522,36.217391,2.913043,27.695652,5.347826,19.956522,5.695652,15.565217,4.782609,15.304348,2.043478,7
2,3,4.383033,0.877892,4.885604,0.898458,51.457584,5.526992,29.917738,5.484576,13.113111,4.377892,8.542416,3.605398,17.742931,2.984576,6
3,4,11.125828,1.576159,14.807947,1.900662,36.132450,3.172185,34.317881,7.000000,32.933775,8.662252,25.576159,11.761589,25.920530,6.880795,8
4,6,14.630682,4.767045,16.625000,5.090909,29.204545,8.113636,25.170455,8.482955,30.636364,11.301136,25.176136,6.528409,30.284091,9.778409,4


In [23]:
# Split train and test
# We use field_ids to split the data to train and test. Note that the test portion for training is different than the test 
# portion provided as part of the competition. 
train_per = 0.8

n_fields = len(df_merged['field_id'])
np.random.seed(10)
train_fields = np.random.choice(df_merged['field_id'], int(n_fields * train_per), replace=False)
test_fields = df_merged['field_id'][~np.in1d(df_merged['field_id'], train_fields)]

In [24]:
X_train, X_test = df_merged[df_merged['field_id'].isin(train_fields)], df_merged[df_merged['field_id'].isin(test_fields)]
X_train = X_train.drop(columns=['label', 'field_id'])
X_test = X_test.drop(columns=['label', 'field_id'])
y_train, y_test = df_merged[df_merged['field_id'].isin(train_fields)]['label'], df_merged[df_merged['field_id'].isin(test_fields)]['label']

In [ ]:
xgb_params = {
    'n_estimators': 500,
    'reg_alpha': .7,
    # 'tree_method':'gpu_hist', # gpu_hist to be used when GPU turned on
    'max_depth': 8,
    'colsample_bytree':0.8
}
xgb_ = xgb.XGBClassifier(**xgb_params,verbosity=1 )
xgb_.fit(X_train, y_train)

In [35]:
pred_proba = xgb_.predict_proba(X_test)

In [36]:
# validation loss
log_loss(y_test, pred_proba)

1.182687285624778

In [37]:
# After running grid search hyperparameter tuning, and concluded to use:
params = {'bootstrap': True,
 'max_depth': 120,
 'max_features': 4,
 'min_samples_leaf': 5,
 'min_samples_split': 10,
 'n_estimators': 300}

In [38]:
# Fitting the RF model
rf = RandomForestClassifier(**params, random_state = 0, n_jobs = -1)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=120, max_features=4,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [39]:
pred_proba_rf = rf.predict_proba(X_test)

In [40]:
# validation loss
log_loss(y_test, pred_proba_rf)

1.197472087051298

## Competition Test Data

In this part we will load the competition test data (which does not have labels) and predict the crop class for each field

In [41]:
tile_ids_test = competition_test_df['tile_id'].unique()

In [42]:
X_competition_test = np.empty((0, 14), np.uint8)
field_ids_test = np.empty((0, 1), np.uint32)


for tile_id in tile_ids_test:
    tile_df = competition_test_df[competition_test_df['tile_id']==tile_id]
    
    field_id_src = rasterio.open(tile_df[tile_df['asset']=='field_ids']['file_path'].values[0])
    field_id_array = field_id_src.read(1).flatten()
    my_test_idx = np.where(field_id_array!=0)[0]
    field_ids_test = np.append(field_ids_test, field_id_array[my_test_idx])
    
    tile_date_times = tile_df[tile_df['satellite_platform']=='s1']['datetime'].unique()
    
    X_tile = np.empty((len(my_test_idx), 0), np.uint8)
              
    if(len(tile_date_times))>36:
      for date_time in tile_date_times[ :40: 6]:
          vv_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='VV')]['file_path'].values[0])
          vv_array = np.expand_dims(vv_src.read(1).flatten()[my_test_idx], axis=1)
          
          vh_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='VH')]['file_path'].values[0])
          vh_array = np.expand_dims(vh_src.read(1).flatten()[my_test_idx], axis=1)
          
          X_tile = np.append(X_tile, vv_array, axis = 1)
          X_tile = np.append(X_tile, vh_array, axis = 1)
    else:
      for date_time in tile_date_times[ :20: 3]:
          vv_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='VV')]['file_path'].values[0])
          vv_array = np.expand_dims(vv_src.read(1).flatten()[my_test_idx], axis=1)
          
          vh_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='VH')]['file_path'].values[0])
          vh_array = np.expand_dims(vh_src.read(1).flatten()[my_test_idx], axis=1)
          
          X_tile = np.append(X_tile, vv_array, axis = 1)
          X_tile = np.append(X_tile, vh_array, axis = 1)
        
    X_competition_test = np.append(X_competition_test, X_tile, axis=0)

In [43]:
X_competition_test.shape

(27572137, 14)

In [44]:
data_test = pd.DataFrame(X_competition_test)
data_test['field_id'] = field_ids_test
data_test.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,field_id
0,8,1,4,1,11,2,11,2,7,5,3,4,27,3,102896
1,3,0,3,1,15,0,13,2,15,2,3,4,27,3,102896
2,3,0,4,1,9,0,7,1,15,2,3,0,27,2,102896
3,6,0,7,2,8,0,5,2,15,2,5,0,33,2,102896
4,5,1,2,1,12,1,25,1,24,4,5,1,35,4,102896


In [45]:
del X_competition_test

In [ ]:
#Memory optimised way: perform group-by on individual column 
GP_DATA = data_test

# create instances of the group_by_atrribute object for each cloumn  
gb_objs_test = [group_by_attribute() for i in range(len(GP_DATA.columns)-1) ]

# perform group-by on individual column by passing column name and dataframe
for i,j in enumerate(GP_DATA.columns[:-1]):
  gb_objs_test[i].compute(GP_DATA,j)

# perform concatination on field_id
from functools import reduce 
df_final = [i.gb for i in gb_objs_test] 

df_merged_test = reduce(lambda left,right: pd.merge(left,right,on='field_id'), df_final)

In [47]:
df_merged_test = df_merged_test.reset_index()

In [48]:
y_competition_prob = rf.predict_proba(df_merged_test.drop(columns=['field_id']))

In [49]:
# In this part we format the DataFrame to have column names and order similar to the sample submission file. 
pred_df = pd.DataFrame(y_competition_prob)
pred_df = pred_df.rename(columns={
  7:'Crop_ID_1',
  2:'Crop_ID_2',
  0:'Crop_ID_3',
  1:'Crop_ID_4',
  8:'Crop_ID_5',
  5:'Crop_ID_6',
  4:'Crop_ID_7',
  6:'Crop_ID_8',
  3:'Crop_ID_9'
})

pred_df['field_id']=df_merged_test['field_id']
pred_df = pred_df[['field_id', 'Crop_ID_1', 'Crop_ID_2', 'Crop_ID_3', 'Crop_ID_4', 'Crop_ID_5', 'Crop_ID_6', 'Crop_ID_7', 'Crop_ID_8', 'Crop_ID_9']]
pred_df

,field_id,Crop_ID_1,Crop_ID_2,Crop_ID_3,Crop_ID_4,Crop_ID_5,Crop_ID_6,Crop_ID_7,Crop_ID_8,Crop_ID_9
0,5,0.001048,0.000000,0.000516,0.009696,0.000000,0.042772,0.000000,0.943997,0.001972
1,10,0.002167,0.149491,0.097280,0.194443,0.137910,0.123892,0.222862,0.020788,0.051167
2,11,0.004871,0.085225,0.093828,0.439713,0.037440,0.169821,0.109271,0.045760,0.014071
3,17,0.012396,0.121238,0.261770,0.177297,0.052882,0.137716,0.130472,0.099076,0.007153
4,18,0.002639,0.084294,0.070540,0.413510,0.090041,0.098374,0.165399,0.047877,0.027325
...,...,...,...,...,...,...,...,...,...,...
35290,122722,0.003238,0.682277,0.019072,0.035698,0.104559,0.054050,0.095638,0.004601,0.000868
35291,122724,0.005655,0.147426,0.132404,0.252480,0.086987,0.145288,0.117645,0.026703,0.085412
35292,122726,0.010275,0.009046,0.020239,0.010620,0.000303,0.111707,0.001866,0.834406,0.001538
35293,122730,0.002362,0.066975,0.104546,0.462595,0.036267,0.110957,0.100462,0.044770,0.071064


In [50]:
# Write the predicted probabilites to a csv for submission
pred_df.to_csv('submission.csv', index=False)

# Public leaderboard 5th position 
![]()
<img src='https://i.imgur.com/1AXyWPz.png' alt='Public leaderboard 5th position ' width='1500'/>